In [ ]:
import os
import numpy as np
import pandas as pd
from Bio import SeqIO
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.patches import Patch
import mpl_stylesheet
import re
import gc
# mpl_stylesheet.banskt_presentation(fontfamily = 'mono', fontsize = 20, colors = 'banskt', dpi = 300)

from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

fullmodel = T5ForConditionalGeneration.from_pretrained("../models/prottrans_t5_xl_u50/").to(device)

#fullmodel.full() if str(device)=='cpu' else fullmodel.half()

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('../models/prottrans_t5_xl_u50', do_lower_case=False, legacy=False)

def sequence_masker(seq, i, j, same_extra_token=False):
    masked_sequence_list = seq.split()
    token_num = 0
    if j<=i:
        print(f"index j={j} must be greater than i={i}")
        raise
    for x in range(i, j):
        if j > len(seq):
            break
        masked_sequence_list[x] = f"<extra_id_{token_num}>"
        if not same_extra_token:
            token_num += 1
    return " ".join(masked_sequence_list)

In [ ]:
# Load one dataset

import json

disprot_file = "../AF2_testset.json"
with open(disprot_file) as infmt:
    disprot_dict = json.load(infmt)

In [ ]:
fullmodel.generation_config

GenerationConfig {
  "_from_model_config": true,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.31.0"
}

In [ ]:
from tqdm import tqdm

#torch.backends.cuda.enable_flash_sdp(True)

protids = list(disprot_dict.keys())
loss_dict = dict()
predmatch_dict = dict()
mask_sizes = [1, 5]
for test_prot in protids:
    print(test_prot)
    if test_prot not in predmatch_dict:
        predmatch_dict[test_prot] = dict()
    if test_prot not in loss_dict:
        loss_dict[test_prot] = dict()
    
    target_seq = disprot_dict[test_prot]['seq']
    input_seq = [" ".join(list(re.sub(r"[UZOB]", "X", target_seq)))]
    true_input = tokenizer(input_seq)
    true_tok = torch.tensor(true_input['input_ids']).to(device)

    for mask_size in mask_sizes:
        print(f"#### Mask size: {mask_size} ####")
               
        loss_sequence = list()
        match_sequence = list()
        for i in tqdm(range(len(target_seq)-mask_size+1)):

            masked_seq = [sequence_masker(input_seq[0], i, i+mask_size)]
            tmp = tokenizer(masked_seq)
            input_ids = torch.tensor(tmp['input_ids']).to(device)
            attention_mask = torch.tensor(tmp['attention_mask']).to(device)
            with torch.no_grad():
                emb  = fullmodel(input_ids=input_ids, labels=true_tok)
                loss = emb.loss.cpu()
                loss_sequence.append(loss.item())
                cpulogits = emb.logits.cpu()
                fastpred = tokenizer.decode(torch.tensor(cpulogits[:,:-1,:].numpy().argmax(-1)[0]), skip_special_tokens=False).replace("<"," <").replace(">","> ")
    #             outputs = fullmodel.generate(input_ids=input_ids, max_length=input_ids.shape[1]+10)
    #             prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    #             if fastpred == prediction:
    #                 print("OK ", end=" ")
                if input_seq[0] == fastpred:
                    match_sequence.append(True)
                else:
                    pred_arr = fastpred.split()
                    seq_arr  = input_seq[0].split()
                    if len(pred_arr) == len(seq_arr):
                        local_match_sequence = list()
                        for j in range(len(pred_arr)):
                            if pred_arr[j] != seq_arr[j]:
                                local_match_sequence.append((j,pred_arr[j], seq_arr[j]))
                    else:
                        print("Mismatch length error")
                        raise
                    match_sequence.append(local_match_sequence)
        predmatch_dict[test_prot][f"aa{mask_size}_match"] = match_sequence
        
        loss_dict[test_prot][f"aa{mask_size}_loss"] = loss_sequence
        torch.cuda.empty_cache()

P37840
#### Mask size: 1 ####


100%|███████████████████████████████████████████████████████████████████████| 140/140 [00:09<00:00, 15.54it/s]


#### Mask size: 5 ####


100%|███████████████████████████████████████████████████████████████████████| 136/136 [00:08<00:00, 16.05it/s]


P04637
#### Mask size: 1 ####


...

100%|███████████████████████████████████████████████████████████████████████| 462/462 [00:52<00:00,  8.83it/s]


P23441
#### Mask size: 1 ####


100%|███████████████████████████████████████████████████████████████████████| 372/372 [00:29<00:00, 12.51it/s]


#### Mask size: 5 ####


100%|███████████████████████████████████████████████████████████████████████| 368/368 [00:29<00:00, 12.48it/s]


In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
ax.imshow(np.array(cpulogits[0]))
plt.show()

In [ ]:
with open("disprot_multi_loss.json", 'w') as outfmt:
    json.dump(loss_dict, outfmt)
with open("disprot_multi_mismatch.json", 'w') as outfmt:
    json.dump(predmatch_dict, outfmt)

In [ ]:

# fig = plt.figure(figsize=(6,6), dpi=100)
# ax = fig.add_subplot(111)
# ax.scatter(np.arange(len(target_seq)), loss_sequence)
# plt.show()

In [ ]:
monomer_file = "monomer_datadict.json"
with open(monomer_file) as infmt:
    monomer_dict = json.load(infmt)

In [ ]:
protids = list(monomer_dict.keys())
loss_dict = dict()
predmatch_dict = dict()
mask_sizes = [1, 5]
for test_prot in protids:
    print(test_prot)
    if test_prot not in predmatch_dict:
        predmatch_dict[test_prot] = dict()
    if test_prot not in loss_dict:
        loss_dict[test_prot] = dict()
    
    target_seq = monomer_dict[test_prot]['seq']
    input_seq = [" ".join(list(re.sub(r"[UZOB]", "X", target_seq)))]
    true_input = tokenizer(input_seq)
    true_tok = torch.tensor(true_input['input_ids']).to(device)

    for mask_size in mask_sizes:
        print(f"#### Mask size: {mask_size} ####")
               
        loss_sequence = list()
        match_sequence = list()
        for i in tqdm(range(len(target_seq)-mask_size+1)):

            masked_seq = [sequence_masker(input_seq[0], i, i+mask_size)]
            tmp = tokenizer(masked_seq)
            input_ids = torch.tensor(tmp['input_ids']).to(device)
            attention_mask = torch.tensor(tmp['attention_mask']).to(device)
            with torch.no_grad():
                emb  = fullmodel(input_ids=input_ids, labels=true_tok)
                loss = emb.loss.cpu()
                loss_sequence.append(loss.item())
                cpulogits = emb.logits.cpu()
                fastpred = tokenizer.decode(torch.tensor(cpulogits[:,:-1,:].numpy().argmax(-1)[0]), skip_special_tokens=False).replace("<"," <").replace(">","> ")
                if input_seq[0] == fastpred:
                    match_sequence.append(True)
                else:
                    pred_arr = fastpred.split()
                    seq_arr  = input_seq[0].split()
                    if len(pred_arr) == len(seq_arr):
                        local_match_sequence = list()
                        for j in range(len(pred_arr)):
                            if pred_arr[j] != seq_arr[j]:
                                local_match_sequence.append((j,pred_arr[j], seq_arr[j]))
                    else:
                        print("Mismatch length error")
                        raise
                    match_sequence.append(local_match_sequence)
        predmatch_dict[test_prot][f"aa{mask_size}_match"] = match_sequence
        
        loss_dict[test_prot][f"aa{mask_size}_loss"] = loss_sequence
        torch.cuda.empty_cache()


    
with open("monomer_multi_loss.json", 'w') as outfmt:
    json.dump(loss_dict, outfmt)
with open("monomer_multi_mismatch.json", 'w') as outfmt:
    json.dump(predmatch_dict, outfmt)

1AE9A
#### Mask size: 1 ####


100%|███████████████████████████████████████████████████████████████████████| 179/179 [00:11<00:00, 15.38it/s]


#### Mask size: 5 ####


100%|███████████████████████████████████████████████████████████████████████| 175/175 [00:11<00:00, 15.07it/s]


1AH7A
#### Mask size: 1 ####


...

100%|███████████████████████████████████████████████████████████████████████| 228/228 [00:15<00:00, 14.89it/s]


1B9WA
#### Mask size: 1 ####


100%|█████████████████████████████████████████████████████████████████████████| 95/95 [00:05<00:00, 15.91it/s]


#### Mask size: 5 ####


100%|█████████████████████████████████████████████████████████████████████████| 91/91 [00:05<00:00, 16.31it/s]
